## Homework

This notebook contains solutions for the [week 3 assignments](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/week_3_data_warehouse).

**Setup**

* Create an external table using the FHV tripdata from year 2019.
* Create a table in BigQuery using the FHV tripdata from year 2019 (do not partition nor cluster this table).

Answer:
``` mysql
-- Create an external table using the FHV tripdata from year 2019
CREATE OR REPLACE EXTERNAL TABLE `dogwood-boulder-375109.nyc_tlc_trips.external_fhv_tripdata`
OPTIONS (
  format = 'CSV',
  uris = ['gs://data-lake-dogwood-boulder-375109/fhv/*.csv.gz']
);

-- Create a table in BigQuery using the FHV tripdata from year 2019
CREATE OR REPLACE TABLE `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata` AS
SELECT * FROM `dogwood-boulder-375109.nyc_tlc_trips.external_fhv_tripdata`;
```

**Question 1. What is the count for the FHV tripdata records for year 2019?**

Answer:

We can just check it in the "DETAILS" section of the table metadata. Alternatively, we can run a query like:

    SELECT     COUNT(*) AS n_rows
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata`;

**Question 2. Write a query to count the number of distinct values in *affiliated_base_number* column for the entire dataset on both the tables. What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?**

Answer:

We can get the answer by highlighting (selecting) the respective queries - estimated amount of data that will be processed by BigQuery is displayed in the top right corner of the query editor.

    -- Number of distinct values in the 'Affiliated_base_number' column
    SELECT     COUNT(DISTINCT(Affiliated_base_number))
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.external_fhv_tripdata`;

    SELECT     COUNT(DISTINCT(Affiliated_base_number))
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata`;

**Question 3. How many records have both NULL *PUlocationID* and *DOlocationID* in the entire dataset?**

Answer:

    -- Number of records with NULL both 'PUlocationID' and 'DOlocationID'
    SELECT     COUNT(*) AS n_rows
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata`
    WHERE      PUlocationID IS NULL
               AND DOlocationID IS NULL;

**Question 4. What is the best strategy to optimize the table if query will always filter by *pickup_datetime* and order by *affiliated_base_number*?**

**Question 5. Implement the optimized solution you chose for question 4. Write a query to retrieve the distinct *affiliated_base_number* between 2019-03-01 and 2019-03-31 (inclusive, based on *pickup_datetime*).

Use the non-partitioned BigQuery table you have created earlier abd note the estimated amount of data that will be processed. Now do the same using the partitioned table. What are those values?**

Answer:

    -- Create a table in BigQuery using the FHV data from year 2019.
    -- Partition by  'pickup_datetime' and cluster by 'Affiliated_base_number'
    CREATE OR REPLACE TABLE `dogwood-boulder-375109.nyc_tlc_trips.partitioned_fhv_tripdata`
    PARTITION BY DATE(pickup_datetime)
    CLUSTER BY Affiliated_base_number
    AS 
    SELECT * FROM `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata`;

    -- Distinct values in the 'Affiliated_base_number' column between 2019-03-01 and 2019-03-31 (inclusive, based on 'pickup_datetime')
    SELECT     DISTINCT(Affiliated_base_number) AS unique_affiliated_base_numbers
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.fhv_tripdata`
    WHERE      pickup_datetime >= '2019-03-01'
               AND pickup_datetime < '2019-04-01';

    SELECT     DISTINCT(Affiliated_base_number) AS unique_affiliated_base_numbers
    FROM       `dogwood-boulder-375109.nyc_tlc_trips.partitioned_fhv_tripdata`
    WHERE      pickup_datetime >= '2019-03-01'
               AND pickup_datetime < '2019-04-01';

**Question 6. Where is the data stored in the External Table you have created?**

**Question 7. Is it true, that you should always cluster your data in BigQuery?**
